In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tqdm import tqdm_notebook

from fastmri_recon.data.fastmri_sequences import Masked2DSequence
from fastmri_recon.models.pdnet import pdnet

Using TensorFlow backend.


In [4]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [5]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [6]:
# generators
AF = 4
train_set = Masked2DSequence(train_path, af=AF, inner_slices=8, rand=True, scale_factor=1e6)
val_set = Masked2DSequence(val_path, af=AF, scale_factor=1e6)

In [7]:
run_params = {
    'n_primal': 5,
    'n_dual': 5,
    'n_iter': 10,
    'n_filters': 32,
}

n_epochs = 10
run_id = f'pdnet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

pdnet_af4_1580292254


In [8]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=100, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
    profile_batch=0,
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end

In [9]:
model = pdnet(lr=1e-3, **run_params)
print(model.summary(line_length=150))

Model: "model"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
kspace_input (InputLayer)                        [(None, 640, None, 1)]           0                                                                   
______________________________________________________________________________________________________________________________________________________
buffer_primal (Lambda)                           (None, 640, None, 5)             0                 kspace_input[0][0]                                
______________________________________________________________________________________________________________________________________________________
mask_input (InputLayer)                          [(None, 640, None)]           

In [10]:
%%time
model.fit(
    train_set, 
    steps_per_epoch=5, 
    epochs=n_epochs,
    validation_data=val_set,
    validation_steps=1,
    verbose=1,
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback,],
)

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/data_utils.py:18: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(filename) as h5_obj:


  ...
    to  
  ['...']
Train for 5 steps
Epoch 1/10


/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/data_utils.py:18: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(filename) as h5_obj:


5/5 [==============================] - 29s 6s/step - loss: 37.2598 - mean_squared_error: 2972.1030 - keras_psnr: 14.6130 - keras_ssim: 0.1673
Epoch 2/10
5/5 [==============================] - 27s 5s/step - loss: 19.5920 - mean_squared_error: 798.5304 - keras_psnr: 16.6481 - keras_ssim: 0.2758
Epoch 3/10
5/5 [==============================] - 26s 5s/step - loss: 27.2697 - mean_squared_error: 2475.0693 - keras_psnr: 18.1443 - keras_ssim: 0.3622
Epoch 4/10
5/5 [==============================] - 28s 6s/step - loss: 16.3294 - mean_squared_error: 681.7512 - keras_psnr: 18.9728 - keras_ssim: 0.3270
Epoch 5/10
5/5 [==============================] - 27s 5s/step - loss: 30.7016 - mean_squared_error: 2167.7847 - keras_psnr: 18.2483 - keras_ssim: 0.3983
Epoch 6/10
5/5 [==============================] - 27s 5s/step - loss: 16.6790 - mean_squared_error: 676.5579 - keras_psnr: 19.2699 - keras_ssim: 0.3400
Epoch 7/10
5/5 [==============================] - 26s 5s/step - loss: 24.9254 - mean_squared_err

In [11]:
# %%time
# # simple overfit trials
# data = next(iter(train_ds))
# model.fit(
#     x=data[0], 
#     y=data[1], 
#     batch_size=data[0][0].shape[0], 
#     epochs=500,
#     verbose=2, 
#     shuffle=False,
# )